# Replay Extraction
## Luke Sullivan

In [8]:
##########################
# Imports
##########################
import pandas as pd
import numpy as np
import cv2
from pytesseract import pytesseract

## Broadcast Download

In [9]:
## yt-dlp

In [10]:
path = "test1.mp4"

## Frame Extraction

In [11]:
def frame_extraction(path):
    cap = cv2.VideoCapture(path)
    frames = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    
    cap.release()
    return frames

### [Figure 1] Extracted Frames (before preprocessing)

In [12]:
def display_frames(frames, num_frames=3):
    if len(frames) < num_frames:
        print(f"Video has less than {num_frames} frames.")
        return
    
    random_frames = sample(frames, num_frames)
    
    fig, axes = plt.subplots(1, num_frames, figsize=(15, 5))
    fig.suptitle(f'{num_frames} Random Frames from Video')
    
    for i, frame in enumerate(random_frames):
        # Convert BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        axes[i].imshow(frame_rgb)
        axes[i].axis('off')
        axes[i].set_title(f'Frame {i+1}')
    
    plt.tight_layout()
    plt.show()

In [ ]:
frames = frame_extraction(path)
display_frames(frames)

## Frame Preprocessing